In [83]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [84]:
df = pd.read_csv('C:/Users/PC/Desktop/mitacs/Projet/ansible/playbooks/Model/model_data.csv')

In [85]:
df

,item,type,LOC_playbook,num_plays,num_tasks_playbook,avg_task_playbook,Num_Imports,Num_Roles,similarity,LOC_play,num_tasks_play,avg_task_play,special_chars_play,LOC_task,task_name_len,special_chars_task,class
0,C:\Users\PC\Desktop\mitacs\Projet\ansible\play...,playbook,0,0,0,0,0,0,0.445803,0,0,0.0,0,0,0,0,Incohesive Playbook
1,C:\Users\PC\Desktop\mitacs\Projet\ansible\play...,playbook,43,1,0,0,0,24,0.000000,0,0,0.0,0,0,0,0,complex playbook
2,C:\Users\PC\Desktop\mitacs\Projet\ansible\play...,playbook,0,0,0,0,0,24,0.000000,0,0,0.0,0,0,0,0,high playbook coupling
3,C:\Users\PC\Desktop\mitacs\Projet\ansible\play...,playbook,43,1,0,0,0,0,0.000000,0,0,0.0,0,0,0,0,Large Playbook
4,Run docker osf adminserver container,task,0,0,0,0,0,0,0.000000,0,0,0.0,0,20,0,0,Large Task
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,C:\Users\PC\Desktop\mitacs\Projet\ansible\play...,playbook,0,0,0,0,0,0,0.000000,0,0,0.0,0,0,0,0,Inconsistent Naming Convention
239,service-guest-image | upload-image | find imag...,task,0,0,0,0,0,0,0.000000,0,0,0.0,0,19,0,0,Large Task
240,service-guest-image | upload-image | find imag...,task,0,0,0,0,0,0,0.000000,0,0,0.0,0,0,82,0,Long Task Name
241,service-guest-image | upload-image | Upload ne...,task,0,0,0,0,0,0,0.000000,0,0,0.0,0,20,0,0,Large Task


In [86]:
# Création d'une correspondance entre chaque "smell code" unique et un label distinct
smells = df['class'].unique()
smell_to_label = {smell: f"cs{i+1}" for i, smell in enumerate(smells)}
df['smell_label'] = df['class'].map(smell_to_label)

In [87]:
mlb = MultiLabelBinarizer()
df['smell_labels'] = df['smell_label'].values.tolist()
df_encoded = mlb.fit_transform(df['smell_labels'])

In [88]:
X = df[['LOC_playbook', 'num_plays', 'num_tasks_playbook', 'avg_task_playbook', 'Num_Imports' , 'Num_Roles' , 'similarity' , 'LOC_play' , 'num_tasks_play' , 'avg_task_play', 'special_chars_play', 'LOC_task', 'task_name_len', 'special_chars_task']]
y = df_encoded

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [89]:
y_train_df = pd.DataFrame(y_train, columns=mlb.classes_)
colonnes_une_classe = [col for col in y_train_df.columns if y_train_df[col].nunique() == 1]
y_train_df = y_train_df.drop(columns=colonnes_une_classe)
y_train = y_train_df.values

In [90]:
y_test_df = pd.DataFrame(y_test, columns=mlb.classes_)
y_test_df = y_test_df.drop(columns=colonnes_une_classe)
y_test = y_test_df.values

In [91]:
# Pour identifier les valeurs NaN
print(X_train.isnull().sum())

LOC_playbook          0
num_plays             0
num_tasks_playbook    0
avg_task_playbook     0
Num_Imports           0
Num_Roles             0
similarity            0
LOC_play              0
num_tasks_play        0
avg_task_play         0
special_chars_play    0
LOC_task              0
task_name_len         0
special_chars_task    0
dtype: int64


In [92]:
model = MultiOutputClassifier(LogisticRegression(max_iter=10000))
model.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression(max_iter=10000))

In [93]:
# Pour identifier les valeurs NaN
print(X_test.isnull().sum())

LOC_playbook          0
num_plays             0
num_tasks_playbook    0
avg_task_playbook     0
Num_Imports           0
Num_Roles             0
similarity            0
LOC_play              0
num_tasks_play        0
avg_task_play         0
special_chars_play    0
LOC_task              0
task_name_len         0
special_chars_task    0
dtype: int64


In [94]:
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8775510204081632
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         0
           3       0.88      0.70      0.78        10
           4       1.00      1.00      1.00        24
           5       1.00      1.00      1.00         6
           6       1.00      0.67      0.80         3
           7       1.00      1.00      1.00         1
           8       0.00      0.00      0.00         0

   micro avg       0.91      0.88      0.90        49
   macro avg       0.62      0.60      0.60        49
weighted avg       0.92      0.88      0.89        49
 samples avg       0.88      0.88      0.88        49



c:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for label_col, coef in zip(y_train_df.columns, model.estimators_):
    print(f"Pour le code smell '{label_col}':")
    for feature_name, coefficient in zip(X.columns, coef.coef_[0]):
        print(f"{feature_name}: {coefficient}")
    print("\n")

In [97]:
with open("resultats.txt", "w") as f:
    for label_col, coef in zip(y_train_df.columns, model.estimators_):
        f.write(f"Pour le code smell '{label_col}':\n")
        for feature_name, coefficient in zip(X.columns, coef.coef_[0]):
            f.write(f"{feature_name}: {coefficient}\n")
        f.write("\n")

In [ ]:
# Ajustez la correspondance pour tenir compte des étiquettes '0', '1', etc.
# au lieu de 'cs1', 'cs2', etc.
adjusted_label_to_smell = {str(i): v for i, v in enumerate(smell_to_label.values())}

with open("resultats.txt", "w") as f:
    for label_col, coef in zip(y_train_df.columns, model.estimators_):
        # Obtenez le nom du code smell à partir du label ajusté
        code_smell_name = adjusted_label_to_smell[label_col]
        f.write(f"Pour le code smell '{code_smell_name}':\n")
        for feature_name, coefficient in zip(X.columns, coef.coef_[0]):
            f.write(f"{feature_name}: {coefficient}\n")
        f.write("\n")


In [82]:
# Sauvegarder le DataFrame dans un fichier CSV
df.to_csv('C:/Users/PC/Desktop/mitacs/Projet/ansible/playbooks/Model/df_labled_smell_new.csv', index=False)